In [10]:
from llama_index.llms.ollama import Ollama
import pandas as pd
from jinja2 import Template
import re
llm = Ollama(model="llama3.1:latest", request_timeout=120.0, json_mode=True, use_gpu=True)

In [11]:
prompt_template = Template(
    """
    <|begin_of_text|>

    <|start_header_id|>user<|end_header_id|>

    Document : {{document}}

    What is the publication date? Output as a structured JSON object with format DD/MM/YYYY.
    <|eot_id|>

    {'publication_date':
    """
)

response = llm.complete(
    """
    CONSEIL MUNICIPAL DU 16 FEVRIER 2023
COMPTE RENDU

Le premier décembre deux mille vingt-deux, les membres du Conseil municipal, convoqués par Mme la
Maire le vingt-deux novembre deux mille vingt-deux, se sont réunis en séance publique, à l'Hôtel de Ville, à
20h30 sous la présidence de Mme la Maire.

Etaient présents :
Mme Hélène DE COMARMOND, Maire
    What is the publication date? Output as a structured JSON object with format DD/MM/YYYY."""
)
print(str(response))

{"publication_date": "16/02/2023"}


In [12]:
df = pd.read_pickle('df_with_regex_chunks.pkl')
df.sample(5)

,doc_id,url,cache,text version,nature,published,entity,entity_type,Text,Gold published date,regex_chunks,regex_chunks_strict
274,6986/9d15ce3e0ae946163e7bbc00a7b7f0ee5c5cc59f_...,https://cuvilly.fr/wp-content/uploads/2024/04/...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,29/03/2024,Cuvilly,Commune,None,None,"[Séance du vendredi 29 mars 2024 à 18h15, Date...","[Séance du vendredi 29 mars 2024 à 18h15, Le v..."
177,3218/93141_819-20fev-cd.pdf,https://www.valdemarne.fr/sites/default/files/...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,20/02/2020,Val-de-Marne,Conseil départemental,recueil des\nactes\nadministratifs\ndu départe...,20/02/2022,"[n° 819 du 20 février 2020, n° 819 du 20 févri...","[n° 819 du 20 février 2020, n° 819 du 20 févri..."
277,1970/d2d71632e392154ebc5fe6f150b8e8ea05342fb5_...,https://www.pau.fr/sites/default/files/media/d...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,31/03/2024,Pau,Commune,None,None,,
62,2983/5edce_PV-conseil-communautaire-6_-02-2023...,https://www.coeurdebrenne.fr/fileadmin/pdf/del...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,06/02/2023,CC Coeur de Brenne,Intercommunalité,COMMUNAUTE DE COMMUNES\nCŒUR DE BRENNE\n1 rue ...,02/03/2023,"[1 rue du Prieuré, Conseil Communautaire du 6 ...","[Conseil Communautaire du 6 février 2023, Date..."
69,2830/b03ce3ff9c291dd865773fe087f6243b68df615c_...,http://www.tours-metropole.fr/sites/default/fi...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,27/02/2023,Tours Métropole Val de Loire,Intercommunalité,PROJET DE DELIBERATION\nSEANCE DU CONSEIL DU 2...,27/02/2023,"[SEANCE DU CONSEIL DU 27 FEVRIER 2023, C 2023/...","[Par délibération du 27 juillet 2020, le Conse..."


In [23]:
def predict_date(document, temperature=0.5):
    input = {'document': ' '.join(document)}
    prompt = prompt_template.render(**input)
    output = llm.complete(prompt, max_tokens = 25, temperature=temperature)
    date = re.findall(r"\d{2}/\d{2}/\d{4}", str(output))
    if date: return date[0]
    else: return str(output)

In [24]:
test = df.sample(10)
print(test['published'])
test['prediction'] = test.apply(lambda x: predict_date(x['regex_chunks']), axis=1)
print(test['prediction'])

32     24/01/2023
297    24/05/2022
76     01/03/2023
291    29/03/2022
241    15/05/2020
80     13/03/2023
212    23/06/2022
412    09/06/2020
275    07/12/2023
246    29/02/2024
Name: published, dtype: object
32     01/02/2023
297    24/05/2022
76     01/10/2021
291    29/03/2022
241    28/08/2019
80     13/03/2023
212    23/06/2022
412    09/06/2020
275    07/08/2015
246    29/02/2024
Name: prediction, dtype: object


In [25]:
df['prediction'] = df.apply(lambda x: predict_date(x['regex_chunks']), axis=1)

In [26]:
df[['published','prediction']]

,published,prediction
0,16/01/2023,16/01/2023
1,25/01/2023,25/01/2023
2,31/01/2023,23/01/2023
3,26/01/2023,26/01/2023
4,16/01/2023,16/01/2023
...,...,...
495,24/01/2024,09/01/2024
496,09/01/2024,09/01/2024
497,22/11/2022,22/11/2022
498,21/12/2023,21/12/2023


In [28]:
accuracy = (df["published"] == df["prediction"]).mean() * 100
print(accuracy)

60.8
